In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 50.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 61.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 781.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.7/752.7 kB 25.9 MB/s eta 0:00:00


In [2]:
import transformers
import pandas as pd
import numpy as np

In [3]:
# load dataset
df = pd.read_csv('2000.csv')

In [4]:
df.head(5)

,id,text,category
0,20604786,This paper uses a new data set to estimate the...,Yes
1,8334260,This study was aimed at testing the hypothesis...,Yes
2,62812273,This paper considers a flexible panel data sam...,Yes
3,21740546,This project describes application of an evide...,Yes
4,154379635,Security regulators and the business press hav...,Yes


In [5]:
label_dict = {'Yes': 1, 'No': 0}
df['label'] = df.category.replace(label_dict)
df.head()

,id,text,category,label
0,20604786,This paper uses a new data set to estimate the...,Yes,1
1,8334260,This study was aimed at testing the hypothesis...,Yes,1
2,62812273,This paper considers a flexible panel data sam...,Yes,1
3,21740546,This project describes application of an evide...,Yes,1
4,154379635,Security regulators and the business press hav...,Yes,1


In [6]:
df['label'].value_counts()

1    1194
0     816
Name: label, dtype: int64

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

In [8]:
df['tokens'] = df.text.apply(tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (568 > 512). Running this sequence through the model will result in indexing errors


In [9]:
len_token = []
for i in range(len(df)):
    len_token.append(len(df.iloc[i]['tokens']['input_ids']))

In [10]:
len_token = []
for i in range(len(df)):
    len_token.append(len(df.iloc[i]['tokens']['input_ids']))

2010


In [11]:
df['len_token'] = pd.DataFrame(len_token, columns = ['len_token'])

In [12]:
df.head(5)

,id,text,category,label,tokens,len_token
0,20604786,This paper uses a new data set to estimate the...,Yes,1,"[input_ids, attention_mask]",128
1,8334260,This study was aimed at testing the hypothesis...,Yes,1,"[input_ids, attention_mask]",271
2,62812273,This paper considers a flexible panel data sam...,Yes,1,"[input_ids, attention_mask]",216
3,21740546,This project describes application of an evide...,Yes,1,"[input_ids, attention_mask]",169
4,154379635,Security regulators and the business press hav...,Yes,1,"[input_ids, attention_mask]",212


In [13]:
len(df[df['len_token']<512])

1934

In [14]:
df = df[df['len_token']<512]

In [15]:
df = df.drop(['id','category','tokens','len_token'], axis=1)  

In [16]:
df['label'].value_counts()

1    1140
0     794
Name: label, dtype: int64

In [17]:
df = pd.concat([df, df[df['label'] == 0][:346]])

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.index.values,
    df.label.values,
    test_size = 0.2,
    random_state = 4,
    stratify = df.label.values
)

df['data_type'] = ['not_set']*df.shape[0]
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_test, 'data_type'] = 'test'
df.groupby(['label','data_type']).count()

text
label data_type      
0     test        326
      train       814
1     test        228
      train       912

In [19]:
df.data_type.value_counts()

train    1726
test      554
Name: data_type, dtype: int64

In [20]:
df_train = df.loc[df['data_type'] == 'train']
df_test = df.loc[df['data_type'] == 'test']

In [21]:
df_train.to_csv('train_Bdoc.csv', index=False)
df_test.to_csv('test_Bdoc.csv', index=False)

In [22]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [23]:
import datasets

In [24]:
dataset = datasets.load_dataset('csv', data_files = {'train':'train_Bdoc.csv', 'test':'test_Bdoc.csv'})
dataset

Using custom data configuration default-7526be128f8da261


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/ec2-user/.cache/huggingface/datasets/csv/default-7526be128f8da261/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'data_type'],
        num_rows: 1726
    })
    test: Dataset({
        features: ['text', 'label', 'data_type'],
        num_rows: 554
    })
})

In [25]:
#text = "After a morning of Thrift Store hunting, a friend and I were thinking of lunch, and he ... " #this sentence is very long, Its length is more than 512. In order to save space, not all of them are shown here
# text = tokenizer.tokenize(text)
# if len(text) > 512:
#     text = text[:512]

In [26]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding= 'max_length', truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [27]:
from transformers import AutoModelForSequenceClassification
checkpoint = "distilbert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bia

In [28]:
import numpy as np
from datasets import load_metric
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [45]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=1)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: data_type, text. If data_type, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1726
  Num E

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.508896,0.873646


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: data_type, text. If data_type, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 554
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=216, training_loss=0.30047307191071687, metrics={'train_runtime': 394.5825, 'train_samples_per_second': 4.374, 'train_steps_per_second': 0.547, 'total_flos': 228638730080256.0, 'train_loss': 0.30047307191071687, 'epoch': 1.0})

In [46]:
model.save_pretrained("CustomModels/dataUse")
tokenizer.save_pretrained("CustomModels/dataUse")

Configuration saved in CustomModels/dataUse/config.json
Model weights saved in CustomModels/dataUse/pytorch_model.bin
tokenizer config file saved in CustomModels/dataUse/tokenizer_config.json
Special tokens file saved in CustomModels/dataUse/special_tokens_map.json


('CustomModels/dataUse/tokenizer_config.json',
 'CustomModels/dataUse/special_tokens_map.json',
 'CustomModels/dataUse/vocab.txt',
 'CustomModels/dataUse/added_tokens.json',
 'CustomModels/dataUse/tokenizer.json')

In [47]:
from transformers import AutoModelForSequenceClassification
load_model = AutoModelForSequenceClassification.from_pretrained("CustomModels/dataUse")
load_tokenizer = AutoTokenizer.from_pretrained("CustomModels/dataUse")

loading configuration file CustomModels/dataUse/config.json
Model config DistilBertConfig {
  "_name_or_path": "CustomModels/dataUse",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "vocab_size": 28996
}

loading weights file CustomModels/dataUse/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at CustomModels/dataU

In [48]:
from transformers import pipeline
my_pipeline = pipeline("text-classification", model=load_model, tokenizer=load_tokenizer)

In [49]:
df_t = pd.read_csv('country_classifications.csv',index_col=1)

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [50]:
df_t = df_t.drop(['Unnamed: 0'], axis=1)  

In [51]:
df_1 = df_t.iloc[:140000]

In [ ]:
df_1.head(5)

In [53]:
### test #######

In [54]:
df_1['tokens'] = df_1.abstract.apply(tokenizer)

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [55]:
len_token=[]
for i in range(len(df_1)):
    len_token.append(len(df_1.iloc[i]['tokens']['input_ids']))

In [56]:
df_1['len_token'] = pd.DataFrame(len_token, columns = ['len_token'])

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [57]:
df_11=df_1[df_1['len_token'] > 512]

In [58]:
len(df_11)

11429

In [59]:
df_12 = df_1[df_1['len_token']<512]

In [ ]:
import datetime
starttime = datetime.datetime.now()
df_12['label_p']= df_12.abstract.apply(my_pipeline)
endtime = datetime.datetime.now()
print(endtime - starttime).seconds



2:42:55.899932


/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


AttributeError: 'NoneType' object has no attribute 'seconds'

In [66]:
df_12['label_p'].isna().value_counts()

False    128466
Name: label_p, dtype: int64

In [74]:
df_12.head(5)

,paper_id,title,abstract,year,doi,venue,journal,mag_field_of_study,places,countries,...,IOT,MSR,PYF,SPM,SXM,UMI,WSM,tokens,len_token,label_p
index,,,,,,,,,,,,,,,,,,,,,
0,159350887,No. 8 Cultural and other relations - Mapping C...,Cultural relations between Central Asian and t...,2018,10.17185/duepublico/47242,NaN,NaN,NaN,"[India, Japan, South Korea, Iran , Turkey, Sou...","['IND', 'IRN', 'JPN', 'KOR', 'TUR']",...,0,0,0,0,0,0,0,"[input_ids, attention_mask]",234,"[{'label': 'LABEL_0', 'score': 0.9831673502922..."
1,159352431,The Ship Owner's Lien on Sub-freights and Pers...,"In the event of default by a head charterer, a...",2014,NaN,NaN,Australian and New Zealand Maritime Law Journal,NaN,"[New Zealand, Australia, Canada ]","['AUS', 'CAN', 'NZL']",...,0,0,0,0,0,0,0,"[input_ids, attention_mask]",164,"[{'label': 'LABEL_0', 'score': 0.9654276967048..."
2,159355456,An update on model Ayush wellness clinic at pr...,The Government of India has presented an expan...,2018,10.15406/ijcam.2018.11.00389,NaN,International Journal of Complementary and Alt...,NaN,"[India , India, Secretariat , Unani, Sidha, In...",['IND'],...,0,0,0,0,0,0,0,"[input_ids, attention_mask]",385,"[{'label': 'LABEL_1', 'score': 0.5357764959335..."
3,18980463,Adult and larval photoreceptors use different ...,Although development of the adult Drosophila c...,2007,10.1101/gad.1565407,Genes & development,Genes & development,NaN,[Spalt],['nf'],...,0,0,0,0,0,0,0,"[input_ids, attention_mask]",304,"[{'label': 'LABEL_0', 'score': 0.9475196003913..."
4,18981358,Rubber hand illusion induced by touching the f...,Background: Studies in animals and humans indi...,2014,10.3389/fnhum.2014.00404,Front. Hum. Neurosci.,NaN,NaN,[],['nf'],...,0,0,0,0,0,0,0,"[input_ids, attention_mask]",383,"[{'label': 'LABEL_1', 'score': 0.9949036836624..."


In [79]:
data_use = []
for index, row in df_12.iterrows():
    if row[263][0]['label'] == 'LABEL_1':
        k = 'Yes'
        data_use.append(k)
    else:
        k = 'No'
        data_use.append(k)

In [80]:
df_12.insert(263, 'data_use', data_use, allow_duplicates=False)

In [81]:
df_12.head(5)

,paper_id,title,abstract,year,doi,venue,journal,mag_field_of_study,places,countries,...,MSR,PYF,SPM,SXM,UMI,WSM,tokens,len_token,data_use,label_p
index,,,,,,,,,,,,,,,,,,,,,
0,159350887,No. 8 Cultural and other relations - Mapping C...,Cultural relations between Central Asian and t...,2018,10.17185/duepublico/47242,NaN,NaN,NaN,"[India, Japan, South Korea, Iran , Turkey, Sou...","['IND', 'IRN', 'JPN', 'KOR', 'TUR']",...,0,0,0,0,0,0,"[input_ids, attention_mask]",234,No,"[{'label': 'LABEL_0', 'score': 0.9831673502922..."
1,159352431,The Ship Owner's Lien on Sub-freights and Pers...,"In the event of default by a head charterer, a...",2014,NaN,NaN,Australian and New Zealand Maritime Law Journal,NaN,"[New Zealand, Australia, Canada ]","['AUS', 'CAN', 'NZL']",...,0,0,0,0,0,0,"[input_ids, attention_mask]",164,No,"[{'label': 'LABEL_0', 'score': 0.9654276967048..."
2,159355456,An update on model Ayush wellness clinic at pr...,The Government of India has presented an expan...,2018,10.15406/ijcam.2018.11.00389,NaN,International Journal of Complementary and Alt...,NaN,"[India , India, Secretariat , Unani, Sidha, In...",['IND'],...,0,0,0,0,0,0,"[input_ids, attention_mask]",385,Yes,"[{'label': 'LABEL_1', 'score': 0.5357764959335..."
3,18980463,Adult and larval photoreceptors use different ...,Although development of the adult Drosophila c...,2007,10.1101/gad.1565407,Genes & development,Genes & development,NaN,[Spalt],['nf'],...,0,0,0,0,0,0,"[input_ids, attention_mask]",304,No,"[{'label': 'LABEL_0', 'score': 0.9475196003913..."
4,18981358,Rubber hand illusion induced by touching the f...,Background: Studies in animals and humans indi...,2014,10.3389/fnhum.2014.00404,Front. Hum. Neurosci.,NaN,NaN,[],['nf'],...,0,0,0,0,0,0,"[input_ids, attention_mask]",383,Yes,"[{'label': 'LABEL_1', 'score': 0.9949036836624..."


In [82]:
df_12.to_csv('result_12.csv')

In [83]:
df_12['data_use'].value_counts()

Yes    99944
No     28522
Name: data_use, dtype: int64

In [84]:
df_2 = df_t.iloc[140001:280000]

In [85]:
df_2['tokens'] = df_2.abstract.apply(tokenizer)

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [97]:
len_token=[]
for i in range(len(df_2)):
    len_token.append(len(df_2.iloc[i]['tokens']['input_ids']))

In [101]:
print(len(len_token))

139999


In [104]:
#df_2['len_token'] = pd.DataFrame(len_token, columns = ['len_token'])

df_2.loc[:,'len_token'] = len_token

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [105]:
df_2.head(5)

,paper_id,title,abstract,year,doi,venue,journal,mag_field_of_study,places,countries,...,HMD,IOT,MSR,PYF,SPM,SXM,UMI,WSM,tokens,len_token
index,,,,,,,,,,,,,,,,,,,,,
140001,143899436,Multi-faceted self-efficacy beliefs as predict...,"Abstract In a longitudinal design, 650 young a...",2007,10.1016/j.paid.2007.05.018,NaN,Personality and Individual Differences,NaN,[],['nf'],...,0,0,0,0,0,0,0,0,"[input_ids, attention_mask]",121
140002,27202509,LARYNGEAL REINNERVATION WITH THE HYPOGLOSSAL N...,This study was performed to determine whether ...,2001,10.1177/000348940111000607,NaN,"Annals of Otology, Rhinology & Laryngology",NaN,[],['nf'],...,0,0,0,0,0,0,0,0,"[input_ids, attention_mask]",563
140003,27202780,The combinatorial guidance activities of draxi...,We have shown that draxin is a repulsive axon ...,2013,10.1016/j.ydbio.2012.11.029,Developmental biology,Developmental biology,NaN,[],['nf'],...,0,0,0,0,0,0,0,0,"[input_ids, attention_mask]",448
140004,27204494,Establishing a Baseline: Community Benefit Spe...,Context:Community Benefit spending by not-for-...,2017,10.1097/PHH.0000000000000493,NaN,Journal of Public Health Management and Practice,NaN,[],['nf'],...,0,0,0,0,0,0,0,0,"[input_ids, attention_mask]",66
140005,27205443,Experience of Long-Term Afala Treatment in Ben...,The use of ultralow doses of antibodies to pro...,2009,10.1007/s10517-009-0695-8,Bulletin of Experimental Biology and Medicine,Bulletin of Experimental Biology and Medicine,NaN,[],['nf'],...,0,0,0,0,0,0,0,0,"[input_ids, attention_mask]",96


In [106]:
df_21=df_2[df_2['len_token'] > 512]

In [213]:
len(df_21)

11320

In [107]:
df_22 = df_2[df_2['len_token']<512]

In [110]:
len(df_22)

128560

In [ ]:
df_22['label_p']= df_22.abstract.apply(my_pipeline)

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [109]:
df_22['label_p'].isna().value_counts()

False    128560
Name: label_p, dtype: int64

In [111]:
df_22.head(5)

,paper_id,title,abstract,year,doi,venue,journal,mag_field_of_study,places,countries,...,IOT,MSR,PYF,SPM,SXM,UMI,WSM,tokens,len_token,label_p
index,,,,,,,,,,,,,,,,,,,,,
140001,143899436,Multi-faceted self-efficacy beliefs as predict...,"Abstract In a longitudinal design, 650 young a...",2007,10.1016/j.paid.2007.05.018,NaN,Personality and Individual Differences,NaN,[],['nf'],...,0,0,0,0,0,0,0,"[input_ids, attention_mask]",121,"[{'label': 'LABEL_1', 'score': 0.9976602792739..."
140003,27202780,The combinatorial guidance activities of draxi...,We have shown that draxin is a repulsive axon ...,2013,10.1016/j.ydbio.2012.11.029,Developmental biology,Developmental biology,NaN,[],['nf'],...,0,0,0,0,0,0,0,"[input_ids, attention_mask]",448,"[{'label': 'LABEL_1', 'score': 0.9979857206344..."
140004,27204494,Establishing a Baseline: Community Benefit Spe...,Context:Community Benefit spending by not-for-...,2017,10.1097/PHH.0000000000000493,NaN,Journal of Public Health Management and Practice,NaN,[],['nf'],...,0,0,0,0,0,0,0,"[input_ids, attention_mask]",66,"[{'label': 'LABEL_0', 'score': 0.9112386703491..."
140005,27205443,Experience of Long-Term Afala Treatment in Ben...,The use of ultralow doses of antibodies to pro...,2009,10.1007/s10517-009-0695-8,Bulletin of Experimental Biology and Medicine,Bulletin of Experimental Biology and Medicine,NaN,[],['nf'],...,0,0,0,0,0,0,0,"[input_ids, attention_mask]",96,"[{'label': 'LABEL_0', 'score': 0.9709614515304..."
140006,27205655,Immuno-identification of cultures of fungi pat...,Because isolates of the fungal pathogensBlasto...,2007,10.1007/BF02601665,Current Microbiology,Current Microbiology,NaN,[],['nf'],...,0,0,0,0,0,0,0,"[input_ids, attention_mask]",228,"[{'label': 'LABEL_1', 'score': 0.9913798570632..."


In [112]:
data_use = []
for index, row in df_22.iterrows():
    if row[263][0]['label'] == 'LABEL_1':
        k = 'Yes'
        data_use.append(k)
    else:
        k = 'No'
        data_use.append(k)

In [113]:
df_22.insert(263, 'data_use', data_use, allow_duplicates=False)

In [114]:
df_22['data_use'].value_counts()

Yes    99482
No     29078
Name: data_use, dtype: int64

In [115]:
df_22.to_csv('result_22.csv')

In [116]:
df_3 = df_t.iloc[280000:500000]

In [147]:
len(df_3)

220000

In [ ]:
df_3['tokens'] = df_3.abstract.apply(tokenizer)

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
len_token=[]
for i in range(len(df_3)):
    len_token.append(len(df_3.iloc[i]['tokens']['input_ids']))

In [156]:
print(len_token[:100])

[192, 177, 182, 263, 371, 281, 353, 285, 246, 196, 107, 355, 36, 155, 108, 259, 190, 150, 192, 44, 452, 230, 589, 117, 360, 140, 250, 398, 228, 724, 163, 159, 466, 66, 500, 148, 160, 456, 244, 258, 184, 98, 542, 285, 457, 123, 514, 284, 437, 249, 221, 255, 476, 113, 332, 133, 280, 510, 202, 178, 211, 69, 55, 288, 515, 296, 391, 152, 413, 948, 190, 300, 691, 183, 381, 341, 301, 375, 262, 346, 219, 185, 212, 118, 346, 446, 403, 268, 301, 445, 117, 275, 271, 153, 43, 150, 245, 178, 176, 34]


In [159]:
df_3.loc[:,'len_token'] = len_token

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [161]:
df_31=df_3[df_3['len_token'] > 512]

In [170]:
len(df_31)

17801

In [175]:
df_32 = df_3[df_3['len_token']<512]

In [177]:
df_long = pd.concat([df_11,df_21,df_31],ignore_index=True)

In [195]:
df_long = pd.concat([df_long,df_41],ignore_index=True)

In [214]:
len(df_long)

56790

In [196]:
df_long.to_csv('long_half.csv')

In [ ]:
df_32['label_p']= df_32.abstract.apply(my_pipeline)

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
df_32['label_p'].isna().value_counts()

False    202001
Name: label_p, dtype: int64

In [ ]:
data_use = []
for index, row in df_32.iterrows():
    if row[263][0]['label'] == 'LABEL_1':
        k = 'Yes'
        data_use.append(k)
    else:
        k = 'No'
        data_use.append(k)

In [ ]:
df_32.insert(263, 'data_use', data_use, allow_duplicates=False)

In [ ]:
df_32['data_use'].value_counts()

Yes    156725
No      45276
Name: data_use, dtype: int64

In [ ]:
df_32.to_csv('result_32.csv')

In [218]:
df_save = pd.concat([df_save,df_42],ignore_index=True)

In [274]:
df_save.to_csv('save_half.csv')

In [187]:
df_4 = df_t.iloc[500000:700000]

In [188]:
len(df_4)

200000

In [189]:
df_4['tokens'] = df_4.abstract.apply(tokenizer)

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [190]:
len_token=[]
for i in range(len(df_4)):
    len_token.append(len(df_4.iloc[i]['tokens']['input_ids']))

In [191]:
df_4.loc[:,'len_token'] = len_token

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [192]:
df_41=df_4[df_4['len_token'] > 512]

In [193]:
len(df_41)

16240

In [194]:
df_42 = df_4[df_4['len_token']<512]

In [ ]:
len(df_42)

183600

In [ ]:
df_42['label_p']= df_42.abstract.apply(my_pipeline)

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
df_42['label_p'].isna().value_counts()

False    183600
Name: label_p, dtype: int64

In [ ]:
data_use = []
for index, row in df_42.iterrows():
    if row[263][0]['label'] == 'LABEL_1':
        k = 'Yes'
        data_use.append(k)
    else:
        k = 'No'
        data_use.append(k)

In [ ]:
df_42.insert(263, 'data_use', data_use, allow_duplicates=False)

In [ ]:
df_42['data_use'].value_counts()

Yes    142778
No      40822
Name: data_use, dtype: int64

In [ ]:
df_42.to_csv('result_42.csv')

In [204]:
df_5 = df_t.iloc[700000:]

In [205]:
len(df_5)

338612

In [206]:
df_5['tokens'] = df_5.abstract.apply(tokenizer)

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [207]:
len_token=[]
for i in range(len(df_5)):
    len_token.append(len(df_5.iloc[i]['tokens']['input_ids']))

In [208]:
df_5.loc[:,'len_token'] = len_token

In [209]:
df_5.head(5)

,paper_id,title,abstract,year,doi,venue,journal,mag_field_of_study,places,countries,...,HMD,IOT,MSR,PYF,SPM,SXM,UMI,WSM,tokens,len_token
index,,,,,,,,,,,,,,,,,,,,,
700001,21072482,Validation of the Japanese version of the qual...,PURPOSE ::: The quality of recovery score QoR-...,2011,10.1007/s00540-011-1151-2,Journal of Anesthesia,Journal of Anesthesia,NaN,[Japan],['JPN'],...,0,0,0,0,0,0,0,0,"[input_ids, attention_mask]",496
700002,21072677,Trends in hepatitis B virus infection among bl...,The objective of this study was to determine t...,2007,NaN,The Southeast Asian journal of tropical medici...,The Southeast Asian journal of tropical medici...,NaN,"[Malaysia, Kelantan, Malaysia]",['MYS'],...,0,0,0,0,0,0,0,0,"[input_ids, attention_mask]",198
700003,21073138,Microscope observations of ART excavated cavit...,This in vitro light and scanning electron micr...,2002,NaN,SADJ : journal of the South African Dental Ass...,SADJ : journal of the South African Dental Ass...,NaN,[],['nf'],...,0,0,0,0,0,0,0,0,"[input_ids, attention_mask]",427
700004,21073322,Transplantation: Pretransplantation serum phos...,Transplantation: Pretransplantation serum phos...,2011,10.1038/nrneph.2011.167,Nature Reviews Nephrology,Nature Reviews Nephrology,NaN,[],['nf'],...,0,0,0,0,0,0,0,0,"[input_ids, attention_mask]",26
700005,21073443,Woodward's Words: Elegant and Commanding,Robert Burns Woodward's publications exhibit a...,2016,10.1002/anie.201600811,Angewandte Chemie,Angewandte Chemie,NaN,[],['nf'],...,0,0,0,0,0,0,0,0,"[input_ids, attention_mask]",64


In [210]:
df_51=df_5[df_5['len_token'] > 512]

In [ ]:
len(df_51)

In [212]:
len(df_save)

459027

In [215]:
df_long = pd.concat([df_long,df_51],ignore_index=True)

In [231]:
len(df_long)

84586

In [217]:
df_long.to_csv('long_half.csv')

In [220]:
df_52 = df_5[df_5['len_token']<512]

In [ ]:
len(df_52)

310535

In [ ]:
df_52['label_p']= df_52.abstract.apply(my_pipeline)

In [ ]:
df_52['label_p'].isna().value_counts()

In [ ]:
data_use = []
for index, row in df_52.iterrows():
    if row[263][0]['label'] == 'LABEL_1':
        k = 'Yes'
        data_use.append(k)
    else:
        k = 'No'
        data_use.append(k)

In [ ]:
df_52.insert(263, 'data_use', data_use, allow_duplicates=False)

In [ ]:
df_52['data_use'].value_counts()

In [ ]:
df_52.to_csv('result_52.csv')

In [254]:
len(df_52)

310535

In [ ]:
paper_rank = df_save['countries'].value_counts().rename_axis('countries_code').reset_index(name='paper_count')

In [ ]:
print(paper_rank)

                                         countries_code  paper_count
0                                                ['nf']       829628
1                                               ['USA']         9476
2                                               ['CHN']         8764
3                                               ['IND']         5224
4                                               ['AUS']         3950
...                                                 ...          ...
7325                       ['AUS', 'CAN', 'ZAF', 'USA']            1
7326                ['HUN', 'ESP', 'SWE', 'GBR', 'USA']            1
7327  ['AUS', 'BRA', 'CAN', 'DNK', 'FIN', 'DEU', 'LT...            1
7328                              ['IND', 'KEN', 'UGA']            1
7329                              ['IRL', 'POL', 'ESP']            1

[7330 rows x 2 columns]


In [276]:
C_count = df_save.loc[:,'ABW':'WSM'].sum().to_frame(name = "papers").sort_values(by = "papers", ascending = False)

In [292]:
len(df_save)

953162

In [279]:
len(C_c)

953162

In [277]:
C_y = df_save.loc[:,'year']

In [278]:
C_c = df_save.loc[:,'ABW':'WSM']

In [280]:
C = pd.concat([C_y,C_c],axis = 1)

In [281]:
print(C_count)

     papers
nf   829628
USA   13576
CHN   12415
IND    7226
AUS    6033
..      ...
HMD       0
IOT       0
SPM       0
SXM       0
UMI       0

[251 rows x 1 columns]


In [291]:
data_yes = df_save[df_save['data_use']=='Yes']

In [293]:
len(data_yes)

739982

In [294]:
DataYes_count = data_yes.loc[:,'ABW':'WSM'].sum().to_frame(name = "Data Yes papers").sort_values(by = "Data Yes papers", ascending = False)

In [295]:
print(DataYes_count)

     Data Yes papers
nf            642763
CHN             9886
USA             9217
IND             5471
AUS             4568
..               ...
ATF                0
SMR                0
SPM                0
SXM                0
UMI                0

[251 rows x 1 columns]


In [298]:
result_update = pd.concat([C_count,DataYes_count], axis=1)

In [266]:
DataYes_count.to_csv('dataYes.csv')

In [267]:
C_count.to_csv('CountryCount.csv')

In [299]:
result_update.to_csv('result_update.csv')

In [255]:
df_save = pd.concat([df_save,df_52],ignore_index=True)

In [1]:
import pandas as pd

In [3]:
long_df = pd.read_csv('long_half.csv')

In [4]:
import transformers

In [5]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
from transformers import AutoModelForSequenceClassification
checkpoint = "distilbert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
#tokenizer = AutoTokenizer.from_pretrained("CustomModels/dataUse")
#model = AutoModelForSequenceClassification.from_pretrained("CustomModels/dataUse")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/251M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.w

In [65]:
txt = long_df['abstract'][99]

In [66]:
print(txt)

Introduction ::: Evidence for linkage and association of obesity-related quantitative traits to chromosome 1q43 has been reported in the Quebec Family Study (QFS) and in populations of Caribbean Hispanic ancestries yet no specific candidate locus has been replicated to date. ::: Methods ::: Using a set of 1,902 single nucleotide polymorphisms (SNPs) genotyped in 525 African American (AA) and 391 European American (EA) women enrolled in the NIEHS uterine fibroid study (NIEHS-UFS), we generated a fine association map for the body mass index (BMI) across a 2.3 megabase-long interval delimited by RGS7 (regulator of G-protein signaling 7) and PLD5 (Phospholipase D, member 5). Multivariable-adjusted linear regression models were fitted to the data to evaluate the association in race-stratified analyses and meta-analysis. ::: Results ::: The strongest associations were observed in a recessive genetic model and peaked in the 3’ end of RGS7 at intronic rs261802 variant in the AA group (p = 1.0 

In [61]:
def chunkslong(txt):  
    
    tokens = tokenizer.encode_plus(txt, add_special_tokens=False,
                               return_tensors='pt')

    #print(len(tokens['input_ids'][0]))
    #tokens
    a = torch.arange(10)
    #a
    torch.split(a, 4)
    input_id_chunks = tokens['input_ids'][0].split(510)
    mask_chunks = tokens['attention_mask'][0].split(510)
    a = torch.cat((torch.Tensor([101]).float(), a.float(), torch.Tensor([102]).float()),0)
    #print(a)
    padding_len = 20 - a.shape[0]
    #padding_len
    if padding_len > 0:
        a = torch.cat([a, torch.Tensor([0] * padding_len).float()])
    # define target chunksize
    # define target chunksize
    chunksize = 512

    # split into chunks of 510 tokens, we also convert to list (default is tuple which is immutable)
    input_id_chunks = list(tokens['input_ids'][0].split(chunksize - 2))
    mask_chunks = list(tokens['attention_mask'][0].split(chunksize - 2))

    # loop through each chunk
    for i in range(len(input_id_chunks)):
        # add CLS and SEP tokens to input IDs
        input_id_chunks[i] = torch.cat([
            torch.tensor([101]).float(), input_id_chunks[i].float(), torch.tensor([102]).float()
        ])
        # add attention tokens to attention mask
        mask_chunks[i] = torch.cat([
            torch.tensor([1]).float(), mask_chunks[i].float(), torch.tensor([1]).float()
        ])
        # get required padding length
        pad_len = chunksize - input_id_chunks[i].shape[0]
        # check if tensor length satisfies required chunk size
        if pad_len > 0:
            # if padding length is more than 0, we must add padding
            input_id_chunks[i] = torch.cat([
                input_id_chunks[i].float(), torch.Tensor([0] * pad_len).float()
            ])
            mask_chunks[i] = torch.cat([
                mask_chunks[i].float(), torch.Tensor([0] * pad_len).float()
            ])

    # check length of each tensor
    for chunk in input_id_chunks:
        (len(chunk))
    # print final chunk so we can see 101, 102, and 0 (PAD) tokens are all correctly placed
   ##chunk
    input_ids = torch.stack(input_id_chunks)
    attention_mask = torch.stack(mask_chunks)

    input_dict = {
        'input_ids': input_ids.long(),
        'attention_mask': attention_mask.int()
    }
    #input_dict
    outputs = model(**input_dict)
    probs = torch.nn.functional.softmax(outputs[0], dim=-1)
    probs = probs.mean(dim=0)
    #probs
    probs = torch.nn.functional.softmax(outputs[0], dim=-1)
    #probs
    probs = probs.mean(dim=0)
    #probs
    winner = torch.argmax(probs).item()
    return ['LABEL_0','LABEL_1'][winner]

In [62]:
chunkslong(txt)

'LABEL_0'

In [63]:
long_df.abstract[825]

'Indoline carbamates, AN680 and AN917 decrease cytokines, TNF-α and IL-6 in peritoneal macrophages activated by lipopolysaccharide (LPS) and in mouse tissues after LPS injection. They prevent nuclear translocation of nuclear factor κB (NF-κB) and activator protein 1. Only AN917 inhibits cholinesterase (ChE) at relevant concentrations. ChE inhibitors decrease NF-κB by activating α7 nicotinic acetylcholine receptors (α7nAChR). The current study compared the effect of rivastigmine, a ChE inhibitor, AN680 and AN917 on ulcerative colitis induced in mice by ingestion of dextran sodium sulfate (4.5%) solution. Rivastigmine (1 mg/kg), AN680 (2.5-10 mg/kg) and AN917 (2-5 mg/kg) were injected subcutaneously once daily for 8 days. Disease severity was assessed by disease activity index (DAI), colonoscopy, colon length and body weight loss, colonic levels of TNF-α, IL-6, IL-1β and myeloid peroxidase (MPO) activity. AN680 (5 mg/kg) reduced DAI, colon shrinkage, weight loss, histopathological signs 

In [64]:
long_df.abstract.iloc[2:1000].apply(chunkslong).value_counts()

LABEL_0    998
Name: abstract, dtype: int64

In [ ]:
long_df['label_p']= long_df.abstract.apply(chunkslong)

In [26]:
long_df['label_p'].value_counts()

LABEL_1    84586
Name: label_p, dtype: int64

In [36]:
long_df = long_df.drop(['Unnamed: 0'],axis=1)

In [40]:
long_df.columns

Index(['paper_id', 'title', 'abstract', 'year', 'doi', 'venue', 'journal',
       'mag_field_of_study', 'places', 'countries',
       ...
       'IOT', 'MSR', 'PYF', 'SPM', 'SXM', 'UMI', 'WSM', 'tokens', 'len_token',
       'label_p'],
      dtype='object', length=264)

In [42]:
data_use = []
for index, row in long_df.iterrows():
    if row[263] == 'LABEL_1':
        k = 'Yes'
        data_use.append(k)
    else:
        k = 'No'
        data_use.append(k)

In [43]:
long_df.insert(263, 'data_use', data_use, allow_duplicates=False)

In [44]:
long_df.head(5)

,paper_id,title,abstract,year,doi,venue,journal,mag_field_of_study,places,countries,...,MSR,PYF,SPM,SXM,UMI,WSM,tokens,len_token,data_use,label_p
0,18985891,Global Morbidity and Mortality of Leptospirosi...,"Background ::: ::: Leptospirosis, a spirochaet...",2015,10.1371/journal.pntd.0003898,PLoS neglected tropical diseases,PLoS neglected tropical diseases,NaN,[Oceania],['nf'],...,0,0,0,0,0,0,"{'input_ids': [101, 24570, 131, 131, 131, 131,...",540,Yes,LABEL_1
1,18988046,Reduced adult endothelial cell EphB4 function ...,Reduced EphB4 expression is observed during ve...,2013,10.1152/ajpcell.00333.2012,American journal of physiology. Cell physiology,American journal of physiology. Cell physiology,NaN,[],['nf'],...,0,0,0,0,0,0,"{'input_ids': [101, 2156, 15776, 1181, 142, 78...",530,Yes,LABEL_1
2,17781384,Search for genes responsible for the remarkabl...,BackgroundZygosaccharomyces bailii is consider...,2015,10.1186/s12864-015-2278-6,BMC Genomics,BMC Genomics,NaN,[ISA1307 ],['nf'],...,0,0,0,0,0,0,"{'input_ids': [101, 24570, 5301, 1183, 12443, ...",590,Yes,LABEL_1
3,17783439,Single- and double-stranded viral RNA generate...,There has been growing interest in the role of...,2014,10.1093/molehr/gau028,Molecular human reproduction,Molecular human reproduction,NaN,"[MIP-1α, MyD88 , MyD88 , MyD88 , MyD88 , MyD88]",['nf'],...,0,0,0,0,0,0,"{'input_ids': [101, 1247, 1144, 1151, 2898, 21...",559,Yes,LABEL_1
4,17784298,Age-Related Macular Degeneration Is Associated...,OBJECTIVE ::: To investigate whether age-relat...,2007,10.1016/j.ophtha.2006.07.045,Ophthalmology,Ophthalmology,NaN,[],['nf'],...,0,0,0,0,0,0,"{'input_ids': [101, 152, 2064, 4538, 8231, 216...",772,Yes,LABEL_1


In [47]:
long_df.loc[:,'ABW':'WSM'].sum().to_frame(name = "dataUseYes").sort_values(by = "dataUseYes", ascending = False)

,dataUseYes
nf,68095
USA,3056
CHN,2177
DEU,1024
IND,1010
...,...
LIE,0
BES,0
WLF,0
TCA,0


In [49]:
df = pd.read_csv('save_half.csv')

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [52]:
df.head(5)

,Unnamed: 0,paper_id,title,abstract,year,doi,venue,journal,mag_field_of_study,places,...,MSR,PYF,SPM,SXM,UMI,WSM,tokens,len_token,data_use,label_p
0,0,159350887,No. 8 Cultural and other relations - Mapping C...,Cultural relations between Central Asian and t...,2018,10.17185/duepublico/47242,NaN,NaN,NaN,"[India, Japan, South Korea, Iran , Turkey, Sou...",...,0,0,0,0,0,0,"{'input_ids': [101, 6651, 4125, 1206, 1970, 31...",234,No,"[{'label': 'LABEL_0', 'score': 0.9831673502922..."
1,1,159352431,The Ship Owner's Lien on Sub-freights and Pers...,"In the event of default by a head charterer, a...",2014,NaN,NaN,Australian and New Zealand Maritime Law Journal,NaN,"[New Zealand, Australia, Canada ]",...,0,0,0,0,0,0,"{'input_ids': [101, 1130, 1103, 1856, 1104, 12...",164,No,"[{'label': 'LABEL_0', 'score': 0.9654276967048..."
2,2,159355456,An update on model Ayush wellness clinic at pr...,The Government of India has presented an expan...,2018,10.15406/ijcam.2018.11.00389,NaN,International Journal of Complementary and Alt...,NaN,"[India , India, Secretariat , Unani, Sidha, In...",...,0,0,0,0,0,0,"{'input_ids': [101, 1109, 2384, 1104, 1726, 11...",385,Yes,"[{'label': 'LABEL_1', 'score': 0.5357764959335..."
3,3,18980463,Adult and larval photoreceptors use different ...,Although development of the adult Drosophila c...,2007,10.1101/gad.1565407,Genes & development,Genes & development,NaN,[Spalt],...,0,0,0,0,0,0,"{'input_ids': [101, 1966, 1718, 1104, 1103, 44...",304,No,"[{'label': 'LABEL_0', 'score': 0.9475196003913..."
4,4,18981358,Rubber hand illusion induced by touching the f...,Background: Studies in animals and humans indi...,2014,10.3389/fnhum.2014.00404,Front. Hum. Neurosci.,NaN,NaN,[],...,0,0,0,0,0,0,"{'input_ids': [101, 24570, 131, 3829, 1107, 35...",383,Yes,"[{'label': 'LABEL_1', 'score': 0.9949036836624..."


In [53]:
df = df.drop(['Unnamed: 0'],axis=1)

In [54]:
df_completed = pd.concat([df,long_df],ignore_index=True)

In [55]:
len(df_completed)

1037748

In [56]:
df_save = df_completed.drop(['tokens', 'len_token','label_p'],axis=1)

In [57]:
df_save.to_csv('results_completed.csv')

In [58]:
df_completed.to_csv('save_half.csv')

In [74]:
df_completed['data_use'].value_counts()

Yes    824568
No     213180
Name: data_use, dtype: int64

In [70]:
df_r1 = df_completed.loc[:,'ABW':'WSM'].sum().to_frame(name = "papers").sort_values(by = "papers", ascending = False)

In [71]:
df_r2 = df_completed[df_completed['data_use'] =='Yes'].loc[:,'ABW':'WSM'].sum().to_frame(name = "Use data papers").sort_values(by = "Use data papers", ascending = False)

In [72]:
result_updated = pd.concat([df_r1,df_r2], axis=1)

In [73]:
result_updated.to_csv('result_update.csv')

In [99]:
df_countries=df_completed.loc[:,'ABW':'WSM']

In [103]:
df_year = df_completed[['year','data_use']]

In [104]:
result_year = pd.concat([df_countries,df_year], axis=1)

In [134]:
result_15 = result_year[['USA','CHN','IND','AUS','JPN','CAN','DEU','BRA','GBR','IRN','ZAF','ITA','NGA','KOR','FRA','year','data_use']]

In [138]:
result_15.to_csv('result_test.csv')

In [135]:
len(result_15)

1037748

In [139]:
result_15.head(5)

,USA,CHN,IND,AUS,JPN,CAN,DEU,BRA,GBR,IRN,ZAF,ITA,NGA,KOR,FRA,year,data_use
0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,2018,No
1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,2014,No
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2018,Yes
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2007,No
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2014,Yes


In [142]:
result_15.groupby('year').sum().to_csv('result_test_15.csv')

In [145]:
result_year[result_year['data_use']=='Yes'][['USA','CHN','IND','AUS','JPN','CAN','DEU','BRA','GBR','IRN','ZAF','ITA','NGA','KOR','FRA','year','data_use']].groupby('year').sum().to_csv('result_data_15.csv')